In [13]:
import numpy as np
import sympy as sp
from sympy import symbols
from IPython.display import Latex, Math, display
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [16]:
inlet_area = 0.25
displayVariableWithUnits('\\sigma',inlet_area,'m^2')
inlet_total_pressure = 1.4*10**5 # Pa
inlet_total_temperature = 900 # K
mass_flow_rate = 6.34 # kg/s
gamma_gas = 1.33
dimensionaless_mass_flow = (mass_flow_rate*(294*inlet_total_temperature)**0.5)/(inlet_total_pressure*inlet_area)
M,gamma,kappa = symbols('M \\gamma \\kappa')
objectivefunction_symbolic = M*(1+((gamma+1)/2)*M**2)**(-(gamma+1)/(2*(gamma-1))) - kappa
display(objectivefunction_symbolic)
objectivefunctionprime_symbolic = objectivefunction_symbolic.diff(M)
display(objectivefunctionprime_symbolic)
step_size_symbolic = objectivefunction_symbolic/objectivefunctionprime_symbolic
ObjectFunctionNumeric = sp.lambdify((M,gamma,kappa),objectivefunction_symbolic)
step_size_numeric = sp.lambdify((M,gamma,kappa),step_size_symbolic)
intial_M = 1
iteration_stop = 100
iteration_count = 0
while abs(ObjectFunctionNumeric(intial_M,gamma_gas,dimensionaless_mass_flow)) > 1e-8:
  iteration_count += 1
  intial_M = intial_M - step_size_numeric(intial_M,gamma_gas,dimensionaless_mass_flow)
  if iteration_count == iteration_stop:
    print('scheme did not converge')
# print(abs(ObjectFunctionNumeric(intial_M,gamma_gas,dimensionaless_mass_flow)))
# print(iteration_count)
displayVariable('M',intial_M)

<IPython.core.display.Math object>

M*(M**2*(\gamma/2 + 1/2) + 1)**((-\gamma - 1)/(2*\gamma - 2)) - \kappa

2*M**2*(-\gamma - 1)*(\gamma/2 + 1/2)*(M**2*(\gamma/2 + 1/2) + 1)**((-\gamma - 1)/(2*\gamma - 2))/((2*\gamma - 2)*(M**2*(\gamma/2 + 1/2) + 1)) + (M**2*(\gamma/2 + 1/2) + 1)**((-\gamma - 1)/(2*\gamma - 2))

<IPython.core.display.Math object>

In [17]:
def MachFlow(machnumber):
  kappa = dimensionaless_mass_flow
  gamma = 1.33
  M = machnumber
  return (gamma)**0.5 * M * (1 + ((gamma-1)*M**2)/2 )**((-gamma-1)/(2*gamma-2)) - kappa
def MachFlowPrime(machnumber):
  gamma = 1.4
  M = machnumber
  first_part = (gamma)**0.5*(1 + ((gamma-1)*M**2)/2 )**((-gamma-1)/(2*gamma-2))
  second_part = (gamma)**0.5 *M*((-gamma-1)/(2*gamma-2))*(1 + ((gamma-1)*M**2)/2 )**((-gamma-1)/(2*gamma-2)-1)*((gamma-1)*M)
  return first_part + second_part
intial_mach_guess = 2
increment_cutoff = 100
increment_count = 0
while abs(MachFlow(intial_mach_guess))>1e-8:
  increment_count += 1
  if increment_count == increment_cutoff:
    print('scheme didn\'t converge')
    displayVariable('e_r',format_scientific(abs(MachFlow(intial_mach_guess))))
    break
  intial_mach_guess = intial_mach_guess - MachFlow(intial_mach_guess)/MachFlowPrime(intial_mach_guess)
final_mach = intial_mach_guess
displayVariable('M_i',final_mach)
displayVariable('e_r',format_scientific(abs(MachFlow(intial_mach_guess))))
displayVariable('i',increment_count)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
def HouseholdP2(x_intial:float,scheme_function,scheme_prime,scheme_double_prime)->float:
  max_iterations = 1000
  iteration_count = 0
  while abs(scheme_function(x_intial)) > 1e-8:
    iteration_count += 1
    x_intial = x_intial - ((2*scheme_function(x_intial))/(scheme_prime(x_intial) - (scheme_prime(x_intial)**2-scheme_function(x_intial)*scheme_double_prime(x_intial))**0.5))
    max_iterations -=1
    if max_iterations ==0:
      print('The scheme didn\'t converge')
      break
    print(iteration_count)
  return x_intial
def Householder(section_supersonic:bool,area_ratio)->float:
  P = 2/2.4
  Q = 1-P
  if section_supersonic==False:
    R = (area_ratio)**2
    a = P**(1/Q)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P+Q*X)**(1/Q) - R*X
    f_prime = lambda X: (P+Q*X)**((1/Q)-1) - R
    f_double_prime = lambda X: P*(P+Q*X)**((1/Q)-2)
    x_final = HouseholdP2(x_intial,f,f_prime,f_double_prime)
    return (x_final)**0.5
  if section_supersonic == True:
    R = (area_ratio)**(2*Q/P)
    a = Q**(1/P)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P*X+Q)**(1/P) - R*X
    f_prime = lambda X: (P*X+Q)**((1/P)-1) - R
    f_double_prime = lambda X: Q*(P*X+Q)**((1/P)-2)
    x_final = abs(HouseholdP2(x_intial,f,f_prime,f_double_prime))
    return 1/(x_final)**0.5
area_ratio = inlet_area / 0.21
displayVariable('M',Householder(True,area_ratio))

1
2


<IPython.core.display.Math object>